In [ ]:
%matplotlib inline
import datetime
import matplotlib.pyplot as plt
import glob
import os
from obspy import read, UTCDateTime, read_inventory
from obspy.signal import PPSD
import warnings
import pandas as pd
import numpy as np
import matplotlib.gridspec as gridspec
from matplotlib.dates import DateFormatter
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
plt.style.use("ggplot")

from msnoise.api import *

In [ ]:
! msnoise db init --tech 1

In [ ]:
# connect to the database
db = connect()

In [ ]:
update_filter(db, ref=1, low=0.1, mwcs_low=0.1, high=1.0, mwcs_high=1.0, mwcs_wlen=12, mwcs_step=4, used=1)
update_filter(db, ref=2, low=1, mwcs_low=1, high=2, mwcs_high=2, mwcs_wlen=4, mwcs_step=1, used=1)
update_filter(db, ref=3, low=2, mwcs_low=2, high=4, mwcs_high=4, mwcs_wlen=4, mwcs_step=1, used=1)
update_filter(db, ref=4, low=4, mwcs_low=4, high=8, mwcs_high=8, mwcs_wlen=2, mwcs_step=1, used=1)


In [ ]:
# ! msnoise db execute "insert into filters (ref, low, mwcs_low, high, mwcs_high, mwcs_wlen, mwcs_step, used) values (1, 0.1, 0.1, 1.0, 1.0, 12.0, 4.0, 1)"
# ! msnoise db execute "insert into filters (ref, low, mwcs_low, high, mwcs_high, mwcs_wlen, mwcs_step, used) values (2, 1, 1, 2, 2, 12.0, 4.0, 1)"
# ! msnoise db execute "insert into filters (ref, low, mwcs_low, high, mwcs_high, mwcs_wlen, mwcs_step, used) values (3, 2, 2, 4, 4, 12.0, 4.0, 1)"
# ! msnoise db execute "insert into filters (ref, low, mwcs_low, high, mwcs_high, mwcs_wlen, mwcs_step, used) values (4, 4, 4, 8, 8, 2.0, 1.0, 1)"

In [ ]:
update_config(db, name="data_folder", value=r"D:\DATA\SDS")
update_config(db, name="response_path", value=r"D:\DATA\RESPONSE")
update_config(db, name="maxlag", value="60")

In [ ]:
# ! msnoise config set resampling_method=Decimate
# ! msnoise config set data_folder="D:/DATA/SDS"
# ! msnoise config set response_path="D:/DATA/RESPONSE"

In [ ]:
! msnoise info

In [ ]:
! msnoise -v populate

In [ ]:
for station in get_stations(db, all=True, format="raw"):
    print(station.net, station.sta)

In [ ]:
! msnoise -v -t 4 scan_archive --init

In [ ]:
! msnoise db update_loc_chan

In [ ]:
from msnoise.plots.data_availability import main as plot_DA

In [ ]:
plot_DA(chan="HH?")

In [ ]:
! msnoise -v new_jobs --init

In [ ]:
! msnoise info -j

In [ ]:
! msnoise -t 5 -d 5 -v cc compute_cc

In [ ]:


# Obtain a list of dates between ``start_date`` and ``enddate``
start, end, datelist = build_movstack_datelist(db)

# Get the list of parameters from the DB:
params = get_params(db)

# Get the time axis for plotting the CCF:
taxis = get_t_axis(db)

filter_id = 1

# Get the results for two station, filter id=1, ZZ component, mov_stack=("1d","1d") and the results as a 2D array:
ccfs = get_results_all(db, "PF.CSS.00", "PF.FOR.00", filter_id, "ZZ", datelist, format="xarray")


In [ ]:
ccfs.CCF.plot()

In [ ]:
zoom = 20 # +-seconds in the coda

In [ ]:
ccfs.CCF.loc[:,-zoom:zoom].plot()

In [ ]:
ccfs.CCF.loc[:,-zoom:zoom].resample(times='12h').mean().plot()

In [ ]:
ccfs.CCF.loc[:,-zoom:zoom].resample(times='12h').median().plot()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(12,5))
print(ax)
ccfs.CCF.loc[:,-zoom:zoom].mean(axis=0).plot(ax=ax,label="mean")
ccfs.CCF.loc[:,-zoom:zoom].median(axis=0).plot(ax=ax, label="median")
plt.legend()


Compute the reference stack and plot vs distance:

In [ ]:
! msnoise config sync
! msnoise cc stack -r

In [ ]:
from msnoise.plots.distance import main as plot_distance
plot_distance(filterid=1, components="ZZ", show=False)
# plt.xlim(-20,20)
plt.show()

reset the STACK jobs and run the mov_stack

In [ ]:
! msnoise reset STACK
! msnoise config set mov_stack=(('1d','1d'),('2d','1d'))


In [ ]:
from msnoise.s04_stack2 import main as stack
# stack("mov", loglevel="DEBUG")

In [ ]:
! msnoise -t 4 cc stack -m

In [ ]:
! msnoise info -j